In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
df=pd.read_csv(r"/content/Reviews.csv.zip")

In [19]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [20]:
df.Score.value_counts()

,count
Score,
5,363122
4,80655
1,52268
3,42640
2,29769


In [21]:
df=df[["Text","Score"]]

In [22]:
df.head()

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


In [23]:
df.isna().sum()

,0
Text,0
Score,0


In [24]:
blanks = []

for index , text in df['Text'].items():
  if text.isspace():
    blanks.append(index)
len(blanks)

0

In [25]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re

lemma = WordNetLemmatizer()

In [26]:
nlp = spacy.load('en_core_web_sm')
nlp = spacy.blank('en')

list1 = nlp.Defaults.stop_words
print("number of spacy ", len(list1))

nltk.download('stopwords')
list2 = set(stopwords.words('english'))
print("number of nltk ", len(list2))

stopwords = set(list1 | list2)
print("number of stopwords ", len(stopwords))

number of spacy  326
number of nltk  198
number of stopwords  401


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
from sklearn.model_selection import train_test_split

x = df['Text']
y= df['Score']

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.33 , random_state=42)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vc = TfidfVectorizer()
x_train_tfidf = vc.fit_transform(x_train)

In [29]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [15]:
text_clf = Pipeline([('tfidf' , TfidfVectorizer()) , ('clf' , LinearSVC())])
text_clf.fit(x_train , y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score , classification_report

In [30]:
def model(model):

  y_pred1 = model.predict(x_test)
  print("accuracy_score :",accuracy_score(y_test , y_pred1))
  print("confusion_matrix :",confusion_matrix(y_test , y_pred1))
  print(classification_report(y_test, y_pred1))


In [31]:
model(text_clf)

accuracy_score : 0.7658297350605043
confusion_matrix : [[ 12402    709    601    431   2900]
 [  2327   2911   1210    768   2481]
 [  1389    755   5108   2196   4666]
 [   654    289   1343   8333  16173]
 [  1066    251    727   2992 114908]]
              precision    recall  f1-score   support

           1       0.70      0.73      0.71     17043
           2       0.59      0.30      0.40      9697
           3       0.57      0.36      0.44     14114
           4       0.57      0.31      0.40     26792
           5       0.81      0.96      0.88    119944

    accuracy                           0.77    187590
   macro avg       0.65      0.53      0.57    187590
weighted avg       0.74      0.77      0.74    187590



In [32]:
text_clf.predict(["This product is amazing, works perfectly and I love it"])

array([5])

In [33]:
from sklearn.model_selection import train_test_split

x = df['Text']
y= df['Score']

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.33 , random_state=42)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

log_reg_model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(solver='lbfgs'))
])

log_reg_model.fit(x_train, y_train)

y_pred_lr = log_reg_model.predict(x_test)

print("Logistic Regression Accuracy:",
      accuracy_score(y_test, y_pred_lr))


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.7418519110826803
